In [ ]:
import wrds

# Connect to WRDS
db = wrds.Connection()

# Define the query
query = """
SELECT 
    msf.permno AS permno,
    msf.date AS date,
    msf.ret AS total_return_incl_dividends,
    msf.retx AS return_excl_dividends,
    msf.prc AS price,
    msf.vol AS volume,
    msf.shrout * 1000 AS shares_outstanding,  -- Convert shares outstanding to actual number
    msf.hexcd AS exchange_code,
    stocknames.ticker AS ticker,
    stocknames.comnam AS company_name,
    (msf.prc * msf.shrout * 1000) AS market_cap  -- Calculate market cap
FROM 
    crsp.msf AS msf
JOIN 
    crsp.stocknames AS stocknames
ON 
    msf.permno = stocknames.permno
WHERE 
    msf.date >= '1963-07-01' 
    AND msf.date <= '2024-08-31'
    AND msf.hexcd IN (1, 2, 3)
    AND stocknames.namedt <= msf.date 
    AND msf.date <= stocknames.nameenddt  -- Corrected column name
ORDER BY 
    msf.permno, msf.date
"""

# Execute the query and fetch the data
crsp_data = db.raw_sql(query)

# Close the WRDS connection
db.close()

# Display the first few rows of the data
crsp_data.head()

crsp_data.to_csv("dataset.csv")